In [19]:
import pandas as pd
# pd.set_option("display.max_rows", None)
# pd.set_option("display.max_columns", None)
# pd.set_option("display.max_colwidth", None)
import yfinance as yf
from transformers import AutoTokenizer, AutoModelForCausalLM,pipeline
import torch
model_id = "tarun7r/Finance-Llama-8B"
torch.cuda.set_device(3)  # Sets default to GPU 0
device=torch.device("cuda:3")
tokenizer = AutoTokenizer.from_pretrained(model_id, trust_remote_code=True)
model = AutoModelForCausalLM.from_pretrained(
    model_id,
    device_map={"": 3},             # auto-distributes across GPUs
    torch_dtype="auto",            # picks bf16 or fp16 depending on availability
)

Loading checkpoint shards: 100%|██████████| 4/4 [00:03<00:00,  1.07it/s]


In [ ]:
def get_financial_info(t):
    ticker = yf.Ticker(t)
    income_stmt = ticker.financials
    balance_sheet = ticker.balance_sheet
    cash_flow = ticker.cashflow
    info = ticker.info
    year=income_stmt.columns[0]
    EPS=income_stmt[year]["Diluted EPS"]
    Net_Income=income_stmt[year]["Net Income"]
    Gross_Profit=income_stmt[year]["Gross Profit"]
    Revenue=income_stmt[year]["Total Revenue"]
    Total_Assets=balance_sheet[year]["Total Assets"]
    Total_Liabilities=balance_sheet[year]["Total Liabilities Net Minority Interest"]
    Shareholders_Equity=balance_sheet[year]["Stockholders Equity"]
    Free_Cash_Flow=cash_flow[year]["Free Cash Flow"]
    Investing_Cash_Flow=cash_flow[year]["Investing Cash Flow"]
    Financing_Cash_Flow=cash_flow[year]["Financing Cash Flow"]
    Operating_Cash_Flow=cash_flow[year]["Operating Cash Flow"]
    P_E=ticker.info['trailingPE']
    ROA=ticker.info['returnOnAssets']
    ROE=ticker.info['returnOnEquity']

    prompt = f"""
        You are a fundamental investment analyst. Analyze the financial performance of {t} and give a recommendation: Strong Buy, Buy, Hold, Sell, or Short. 
        Justify your decision in 4–6 bullet points using financial reasoning. Consider all the financial information shared. Only use the numerical data given. 
        Do not add assumptions about company operations, reputation, or strategy.


        Financials for {t}:

        Income Statement:
        - Revenue: ${Revenue:,.0f}
        - Gross Profit: ${Gross_Profit:,.0f}
        - Net Income: ${Net_Income:,.0f}
        - EPS (Diluted): {EPS:.2f}

        Balance Sheet:
        - Total Assets: ${Total_Assets:,.0f}
        - Total Liabilities: ${Total_Liabilities:,.0f}
        - Shareholders' Equity: ${Shareholders_Equity:,.0f}

        Cash Flow:
        - Operating Cash Flow: ${Operating_Cash_Flow:,.0f}
        - Free Cash Flow: ${Free_Cash_Flow:,.0f}
        - Investing Cash Flow: ${Investing_Cash_Flow:,.0f}
        - Financing Cash Flow: ${Financing_Cash_Flow:,.0f}

        Valuation and Ratios:
        - P/E Ratio: {P_E:.2f}
        - ROA: {ROA:.2%}
        - ROE: {ROE:.2%}

        Based on this, what is your investment recommendation? Pick one action candidate.
        """
    return prompt

In [ ]:
prompt=get_financial_info("AAPL")
generator = pipeline(
    "text-generation",
    model=model,
    tokenizer=tokenizer
)

outputs = generator(
    prompt,
    max_new_tokens=500,         # Reduced for memory efficiency
    do_sample=True,
    temperature=0.7,
    top_p=0.9,
    pad_token_id=tokenizer.eos_token_id,
    # Memory efficient generation settings
    num_beams=1,                # No beam search to save memory
    #early_stopping=True,
    use_cache=True
)

generated_text = outputs[0]['generated_text']
print(generated_text)

Device set to use cuda:3



        You are a fundamental investment analyst. Analyze the financial performance of AAPL and give a recommendation: Strong Buy, Buy, Hold, Sell, or Short. 
        Justify your decision in 4–6 bullet points using financial reasoning. Consider all the financial information shared. Only use the numerical data given. 
        Do not add assumptions about company operations, reputation, or strategy.


        Financials for AAPL:

        Income Statement:
        - Revenue: $391,035,000,000
        - Gross Profit: $180,683,000,000
        - Net Income: $93,736,000,000
        - EPS (Diluted): 6.08

        Balance Sheet:
        - Total Assets: $364,980,000,000
        - Total Liabilities: $308,030,000,000
        - Shareholders' Equity: $56,950,000,000

        Cash Flow:
        - Operating Cash Flow: $118,254,000,000
        - Free Cash Flow: $108,807,000,000
        - Investing Cash Flow: $2,935,000,000
        - Financing Cash Flow: $-121,983,000,000

        Valuation and Ratios

In [22]:
tickers = ["AAPL", "MSFT","RDDT","NVDA"]

data = yf.download(tickers, period='1y')
data


/tmp/ipykernel_2356765/1044327187.py:3: FutureWarning: YF.download() has changed argument auto_adjust default to True
  data = yf.download(tickers, period='1y')
[*********************100%***********************]  4 of 4 completed


Price            Close                                            High  \
Ticker            AAPL        MSFT        NVDA        RDDT        AAPL   
Date                                                                     
2024-07-19  223.265839  433.784180  117.892754   64.699997  225.744253   
2024-07-22  222.917496  439.569855  123.500984   68.129997  226.719706   
2024-07-23  223.962601  441.465332  122.551277   67.839996  225.883625   
2024-07-24  217.522720  425.636658  114.213913   62.950001  223.753591   
2024-07-25  216.477615  415.216553  112.244545   62.060001  219.821976   
...                ...         ...         ...         ...         ...   
2025-07-14  208.619995  503.019989  164.070007  146.380005  210.910004   
2025-07-15  209.110001  505.820007  170.699997  144.800003  211.889999   
2025-07-16  210.160004  505.619995  171.369995  144.309998  212.399994   
2025-07-17  210.020004  511.700012  173.000000  142.520004  211.800003   
2025-07-18  211.179993  510.049988  172.410004  148.240005  211.789993   

Price                                                  Low              \
Ticker            MSFT        NVDA        RDDT        AAPL        MSFT   
Date                                                                     
2024-07-19  437.783546  121.561593   68.269997  222.240634  428.713074   
2024-07-22  441.217228  124.030816   70.459999  222.051535  435.570519   
2024-07-23  444.978406  124.650620   71.410004  221.643445  439.728647   
2024-07-24  438.120958  119.912110   66.300003  216.119295  424.336627   
2024-07-25  426.529809  116.593170   64.360001  213.620965  414.333340   
...                ...         ...         ...         ...         ...   
2025-07-14  503.970001  165.490005  150.389999  207.539993  501.029999   
2025-07-15  508.299988  172.399994  148.050003  208.919998  502.790009   
2025-07-16  506.720001  171.750000  146.190002  208.639999  501.890015   
2025-07-17  513.369995  174.160004  147.050003  209.589996  505.619995   
2025-07-18  514.640015  174.250000  149.089996  209.699997  507.429993   

Price                                     Open                          \
Ticker            NVDA        RDDT        AAPL        MSFT        NVDA   
Date                                                                     
2024-07-19  117.332933   64.500000  223.773474  429.804711  120.311987   
2024-07-22  119.822146   65.544998  225.953286  438.428611  120.311989   
2024-07-23  122.061434   67.010002  223.325580  440.522548  122.741220   
2024-07-24  113.404171   62.181000  222.957311  437.098796  119.132357   
2024-07-25  106.266438   59.730000  217.910900  425.537417  113.004307   
...                ...         ...         ...         ...         ...   
2025-07-14  162.020004  145.300003  209.929993  501.519989  165.369995   
2025-07-15  169.199997  142.899994  209.220001  503.019989  171.190002   
2025-07-16  168.899994  138.300003  210.300003  505.179993  171.059998   
2025-07-17  170.830002  140.889999  210.570007  505.679993  172.020004   
2025-07-18  171.259995  138.110001  210.869995  514.479980  173.639999   

Price                     Volume                                
Ticker            RDDT      AAPL      MSFT       NVDA     RDDT  
Date                                                            
2024-07-19   65.750000  49151500  20940400  217223800  1997300  
2024-07-22   66.930000  48201800  15808800  258068900  3644800  
2024-07-23   67.800003  39960300  13107100  173911000  2486500  
2024-07-24   65.665001  61777600  26805800  327776900  2869100  
2024-07-25   62.900002  51391200  29943800  460067000  2644500  
...                ...       ...       ...        ...      ...  
2025-07-14  146.000000  38840100  12058800  136975800  4438700  
2025-07-15  147.289993  42296300  14927200  230627400  4426500  
2025-07-16  144.949997  47490500  15154400  158831500  5853200  
2025-07-17  144.300003  48068100  17503100  160841100  4454700  
2025-07-18  142.509995  48939500  21203400  

In [23]:
def compute_rsi(close, period=14):
    delta = close.diff()

    gain = delta.clip(lower=0)
    loss = -delta.clip(upper=0)

    avg_gain = gain.rolling(window=period).mean()
    avg_loss = loss.rolling(window=period).mean()

    rs = avg_gain / avg_loss
    rsi = 100 - (100 / (1 + rs))

    return rsi

def compute_obv(close, volume):
    direction = close.diff().apply(lambda x: 1 if x > 0 else (-1 if x < 0 else 0))
    obv = (volume * direction).fillna(0).cumsum()
    return obv


In [24]:
data['Close']['AAPL']
temp=pd.DataFrame()
multi_data=[]
for x in tickers:
    temp=pd.DataFrame()
    temp["SMA_5"] = data["Close"][x].rolling(5).mean()
    temp["SMA_15"] = data["Close"][x].rolling(15).mean()
    temp["SMA_50"] = data["Close"][x].rolling(50).mean()

    temp['EMA_5'] = data['Close'][x].ewm(span=5).mean()
    temp['EMA_10'] = data['Close'][x].ewm(span=10).mean()
    temp['EMA_50'] = data['Close'][x].ewm(span=50).mean()
    temp["Date"] = data['Close'].index
    temp["Ticker"] = x
    temp["RSI"]=compute_rsi(data['Close'][x])
    temp["OBV"]=compute_obv(data['Close'][x], data['Volume'][x])
    multi_data.append(temp)
indicators=pd.concat(multi_data)
indicators.set_index(["Ticker", "Date"], inplace=True)
indicators

SMA_5      SMA_15      SMA_50       EMA_5      EMA_10  \
Ticker Date                                                                     
AAPL   2024-07-19         NaN         NaN         NaN  223.265839  223.265839   
       2024-07-22         NaN         NaN         NaN  223.056833  223.074250   
       2024-07-23         NaN         NaN         NaN  223.485881  223.431361   
       2024-07-24         NaN         NaN         NaN  221.008876  221.484727   
       2024-07-25  220.829254         NaN         NaN  219.269387  220.047322   
...                       ...         ...         ...         ...         ...   
NVDA   2025-07-14  163.194003  157.416001  140.625915  162.965192  160.260295   
       2025-07-15  165.334003  159.184668  141.807870  165.543460  162.158423   
       2025-07-16  167.032001  160.749335  142.945429  167.485639  163.833254   
       2025-07-17  168.812000  161.995335  144.129187  169.323759  165.499935   
       2025-07-18  170.310001  163.154668  145.306745  170.352507  166.756311   

                       EMA_50        RSI         OBV  
Ticker Date                                           
AAPL   2024-07-19  223.265839        NaN           0  
       2024-07-22  223.088184        NaN   -48201800  
       2024-07-23  223.391391        NaN    -8241500  
       2024-07-24  221.835044        NaN   -70019100  
       2024-07-25  220.676160        NaN  -121410300  
...                       ...        ...         ...  
NVDA   2025-07-14  143.360036  79.987979  2632260400  
       2025-07-15  144.432246  81.596482  2862887800  
       2025-07-16  145.488680  78.114728  3021719300  
       2025-07-17  146.567607  78.758823  3182560400  
       2025-07-18  147.581080  75.171740  3036452800  

[1000 rows x 8 columns]

In [26]:
def create_technical_prompt(df,t):
    latest = df.loc[t].iloc[-1]
    prompt = f"""
    You are a technical investment analyst. Analyze the recent technical performance of {t} and give a recommendation: Strong Buy, Buy, Hold, Sell, or Short. 
    Justify your decision in 4–6 bullet points using technical analysis. Only use the numerical data given. 
    Do not add assumptions about company fundamentals, operations, or strategy.

    Technical Indicators for {t} (most recent data point):

    - SMA 5: {latest['SMA_5']:.2f}
    - SMA 15: {latest['SMA_15']:.2f}
    - SMA 50: {latest['SMA_50']:.2f}

    - EMA 5: {latest['EMA_5']:.2f}
    - EMA 10: {latest['EMA_10']:.2f}
    - EMA 50: {latest['EMA_50']:.2f}

    - RSI: {latest['RSI']:.2f}
    - OBV: {latest['OBV']:,.0f}

    Based on this, what is your investment recommendation? Pick one action candidate.
    """
    return prompt


In [28]:
ta_prompt=create_technical_prompt(indicators,"AAPL")
ta_prompt

'\n    You are a technical investment analyst. Analyze the recent technical performance of AAPL and give a recommendation: Strong Buy, Buy, Hold, Sell, or Short. \n    Justify your decision in 4–6 bullet points using technical analysis. Only use the numerical data given. \n    Do not add assumptions about company fundamentals, operations, or strategy.\n\n    Technical Indicators for AAPL (most recent data point):\n\n    - SMA 5: 209.82\n    - SMA 15: 209.59\n    - SMA 50: 204.34\n\n    - EMA 5: 210.37\n    - EMA 10: 209.75\n    - EMA 50: 206.55\n\n    - RSI: 70.07\n    - OBV: 551,218,300\n\n    Based on this, what is your investment recommendation? Pick one action candidate.\n    '